In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# Coordinate of KDFW
KDFW = [-96.973496106, 32.584330996] # lon, lat

# WXR region of interest, check the _plotting_and_roi.ipynb for details
# WX_ROI = [(-103, -90.5), (26.5, 38)] # lon, lat
WX_ROI = [(KDFW[0] - 3, KDFW[0] + 3), (KDFW[1] - 3, KDFW[1] + 3)] # lon, lat

ROI_RADIUS = 200 # in km

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert coordinates from degrees to radians
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c
    return distance # in kilometers


# Dropping functions

In [ ]:
def add_distance_col_to_df(df):
    df['distance'] = haversine(df['latitude'], df['longitude'], KDFW[1], KDFW[0]) # KDFW[1] is latitude, KDFW[0] is longitude

# Sample usage: add_distance_col_to_df(df) / inline modification

In [ ]:
def drop_callsigns_not_far_enough_away(df):
    # Compute the max distance for each callsign
    max_distance_per_callsign = df.groupby('callsign')['distance'].max()

    # Callsigns that are within the radius
    callsigns_to_drop = max_distance_per_callsign[max_distance_per_callsign < ROI_RADIUS].index
    print('There are {} callsigns within the radius to be dropped'.format(len(callsigns_to_drop)))

    # Drop callsigns that are within the radius
    return df[~df['callsign'].isin(callsigns_to_drop)]

# Sample usage: df = drop_callsigns_not_far_enough_away(df) / copy

In [ ]:
def drop_callsigns_yet_landed(df):
    # Compute the max distance for each callsign
    min_distance_per_callsign = df.groupby('callsign')['distance'].min()

    # Callsigns that are within the radius
    callsigns_to_drop_2 = min_distance_per_callsign[min_distance_per_callsign > 30].index # 4km within the airport is considered as at the airport
    print('There are {} callsigns that haven\'t landed and was dropped'.format(len(callsigns_to_drop_2)))

    # Drop callsigns that are within the radius
    return df[~df['callsign'].isin(callsigns_to_drop_2)]

# Trimming functions

In [ ]:
def trim_df_to_roi_radius(df):
    return df[df['distance'] <= ROI_RADIUS]


In [ ]:
def trim_and_resample(df, callsign, desired_length = 2000):
    df = df[df['callsign'] == callsign]
    # convert df['timestamp'] to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.set_index('timestamp')
    # fill NaN values
    df.fillna(method='ffill', inplace=True)
    df = df.resample('1s').agg({
        'callsign': 'ffill',
        'groundspeed': 'mean',
        'latitude': 'mean',
        'longitude': 'mean',
        'altitude': 'mean',
        'track': 'mean',
        'vertical_rate': 'mean'
    })
    df = df.reset_index()
    # df = df.interpolate()
    if len(df) > desired_length:
        # trim the trajectory to desired length
        df = df.iloc[0:desired_length]
    elif len(df) < desired_length:
        # repeat the last row until desired length is reached
        last_row = df.iloc[-1]
        while len(df) < desired_length:
            df = pd.concat([df, last_row.to_frame().T])
    return df

# Callsign thunderstorm attribution

In [ ]:
import datetime

# Rounding function
def round_to_nearest_half_hour(ts):
    # Extract minutes
    minutes = ts.minute
    # Determine if we should round up or down
    if minutes < 30:
        return ts.replace(minute=0, second=0, microsecond=0)
    else:
        return ts.replace(minute=30, second=0, microsecond=0)

In [ ]:
def get_roi_entrance_time(df):
    callsign_roi_entrance_time = df.groupby('callsign')['timestamp'].min().reset_index()
    # Convert timestamp strings to datetime
    callsign_roi_entrance_time['timestamp'] = pd.to_datetime(callsign_roi_entrance_time['timestamp'])
    callsign_roi_entrance_time['rounded_timestamp'] = callsign_roi_entrance_time['timestamp'].apply(round_to_nearest_half_hour)

    # Convert rounded_timestamp back to string
    callsign_roi_entrance_time['rounded_timestamp'] = callsign_roi_entrance_time['rounded_timestamp'].dt.strftime('%Y-%m-%d %H_%M_%S')

    return callsign_roi_entrance_time

# Preprocessing of Trajectories (Main Entry Point)

In [ ]:
# surpress warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!cp drive/MyDrive/hrrr17t.tgz hrrr17t.tgz

In [ ]:
!mkdir -p tx
!tar -xzf hrrr17t.tgz -C tx

**Start here**

In [ ]:
traj_dir = "tx/tx/"
# Find all CSV files in the directory
import glob
traj_files = glob.glob(traj_dir + "*.csv.gz")
print("Found {} CSVGZ files in {}".format(len(traj_files), traj_dir))

Found 337 CSVGZ files in tx/tx/


In [ ]:
!mkdir -p stx

In [ ]:
for csv_file in traj_files[42:]:
    # try:
    df = pd.read_csv(csv_file, compression='gzip', error_bad_lines=False)
    df = df[['callsign', 'groundspeed', 'timestamp', 'latitude', 'longitude', 'altitude', 'track', 'vertical_rate']]
    # Distance preprocessing
    add_distance_col_to_df(df)
    df = drop_callsigns_not_far_enough_away(df)
    df = drop_callsigns_yet_landed(df)
    df = trim_df_to_roi_radius(df)
    roi_entrance_time = get_roi_entrance_time(df)
    storm_idents = roi_entrance_time['rounded_timestamp'].unique().tolist()
    import os
    desired_length = 2000

    for storm_ident in storm_idents:
        print('Processing storm {}'.format(storm_ident))
        callsign_of_storm = roi_entrance_time[roi_entrance_time['rounded_timestamp'] == storm_ident]['callsign'].tolist()

        if len(callsign_of_storm) > 20: # only keep 20 callsigns for each storm
          # sample randomly 20 callsigns
          callsign_of_storm = np.random.choice(callsign_of_storm, size=20, replace=False)

        big_trajectory = np.empty((0, 2, desired_length))
        for callsign in callsign_of_storm:
            processed_trajectory_df = trim_and_resample(df, callsign, desired_length)
            # create a new np array
            processed_trajectory = processed_trajectory_df[['latitude', 'longitude']].to_numpy().T.reshape(1, 2, -1)
            # concatenate to big_trajectory
            big_trajectory = np.concatenate((big_trajectory, processed_trajectory), axis=0)

        print('There are {} trajectories to be admitted'.format(big_trajectory.shape[0]))
        # save the big_trajectory
        np.savez_compressed(os.path.join('stx', storm_ident), big_trajectory)

    # except Exception as e:
    #     print(e)
    #     print('Error processing {}'.format(csv_file))

Streaming output truncated to the last 5000 lines.
Processing storm 2017-10-04 23_30_00
There are 6 trajectories to be admitted
Processing storm 2017-10-04 15_30_00
There are 1 trajectories to be admitted
Processing storm 2017-10-05 00_30_00
There are 3 trajectories to be admitted
Processing storm 2017-10-03 21_00_00
There are 3 trajectories to be admitted
Processing storm 2017-10-03 16_00_00
There are 2 trajectories to be admitted
Processing storm 2017-10-04 21_00_00
There are 1 trajectories to be admitted
Processing storm 2017-10-03 17_30_00
There are 9 trajectories to be admitted
Processing storm 2017-10-03 12_00_00
There are 5 trajectories to be admitted
Processing storm 2017-10-03 23_00_00
There are 3 trajectories to be admitted
Processing storm 2017-10-03 18_00_00
There are 2 trajectories to be admitted
Processing storm 2017-10-03 13_00_00
There are 3 trajectories to be admitted
Processing storm 2017-10-03 22_30_00
There are 2 trajectories to be admitted
Processing storm 2017-10-

In [ ]:
csv_file

'tx/tx/hrrr17x_255.csv.gz'

In [ ]:
traj_files.index(csv_file)

336

In [ ]:
# !rm -rf stx/*

In [ ]:
!tar -czf stx17.tar.gz stx/

In [ ]:
!cp stx17.tar.gz /content/drive/MyDrive/stx17.tar.gz